### Base Function Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil.relativedelta import relativedelta
import yfinance as yf
import os

In [2]:
# function to take a list of tickers and prepare a 5-year portfolio of price data
def port_prep(ticker_lst):
    # find dates for import: 5 years of price data
    today = datetime.today().strftime("%Y-%m-%d")
    five = (datetime.today() - relativedelta(years=5)).strftime("%Y-%m-%d")
    
    # remove any blank ticker positions at end of list
    while ticker_lst[-1] == "":
        ticker_lst.pop()
    
    # create class to convert string to object location name
    class classthing:
        def __init__(self, name):
            self.name = name

    # dictionary to hold ticker and pd.series object location name        
    dct = {name: classthing(name) for name in ticker_lst}
    
    # get price data for tickers in list using yfinance where pd.series are memory location names
    for stk in ticker_lst:
        dct[stk] = yf.download(stk, start=five, end=today)['Adj Close']
        
    # combine the four stocks into a single dataframe
    # axis =1 to show that the concatenation is to be done on columns not rows
    port = pd.concat([dct[x] for x in ticker_lst], axis=1)
    # ticker names to portfolio
    port.columns = ticker_lst
    
    return port

In [3]:
# function behind button displaying performance percentages
def chart_perf(ticker_lst):
    # prepare 5-year portfolio from ticker list
    port = port_prep(ticker_lst)
    
    ## RETURN GRAPHING SECTION
    # dataframe based on the stock prices to compute the daily returns.
    port_returns = port.pct_change(1).dropna()
    
    # now combining all four into one step
    cumul_return2 = 100 * ((1 + port_returns).cumprod() - 1)

    # display chart
    fig = plt.figure(figsize=(12,6))
    plt.plot(cumul_return2)
    plt.ylabel('Returns')
    plt.xlabel('Date')
    plt.title('Individual Percent Returns')
    plt.legend(cumul_return2.columns)
    plt.show()

In [4]:
# function behind button to run monte carlo simulation
def mc_hammer(ticker_lst):
    # remove any previous result set
    os.remove('mc.pickle')
    
    # prepare 5-year portfolio from ticker list
    port = port_prep(ticker_lst)
    
    # get the number of stocks in portfolio
    N = len(port.columns)
    
    # calculate the variables just to make this readable
    log_rets = np.log(port / port.shift(1))
    log_rets_cov = log_rets.cov()

    # function to generate random portfolio weights
    def gen_weights(N):
        weights = np.random.random(N)
        return weights / np.sum(weights) 
    
    # function to calculate annualized portfolio return given weighting 
    def calculate_returns(weights, log_rets):
        # annualize return
        return np.sum(log_rets.mean() * weights) * 252
    
    # function to calculate annual portfolio volatility given weighting
    def calculate_volatility(weights, log_rets_cov):
        annualized_cov = np.dot(log_rets.cov() * 252, weights)
        vol = np.dot(weights.transpose(), annualized_cov)
        return np.sqrt(vol)
    
    """
        # Monte Carlo Simulation
    """
    # lists to hold the results from the monte carlo simulation
    mc_weights = []
    mc_portfolio_returns = []
    mc_portfolio_vol = []

    # note that the number of stocks in the portfolio makes the length of
    # the simulation grow exponentially in complexity
    for sim in range(6000):

        weights = gen_weights(N)
        mc_weights.append(weights)
        sim_returns = calculate_returns(weights, log_rets)
        mc_portfolio_returns.append(sim_returns)
        sim_volatility = calculate_volatility(weights, log_rets_cov)
        mc_portfolio_vol.append(sim_volatility)
        
    # assume zero risk-free rate to calculate Sharpe ratio
    mc_sharpe_ratios = np.array(mc_portfolio_returns / np.array(mc_portfolio_vol))
    
    port_list = list(port.columns)
    
    # assemble dataframe of results
    df_weights = pd.DataFrame(np.row_stack(mc_weights))
    df_weights.columns = port_list
    
    df_portfolio_returns = pd.DataFrame(np.row_stack(mc_portfolio_returns))
    df_portfolio_returns.columns = ['Return']
    
    df_portfolio_vol = pd.DataFrame(np.row_stack(mc_portfolio_vol))
    df_portfolio_vol.columns = ['Volatility']
    
    df_portfolio_sharpe = pd.DataFrame(np.row_stack(mc_sharpe_ratios))
    df_portfolio_sharpe.columns = ['Sharpe']
    
    df_portfolio_mc = pd.concat([df_weights, df_portfolio_returns, df_portfolio_vol, df_portfolio_sharpe], axis=1)
    
    # sort dataframe descending by Sharpe ratio
    mc_final = df_portfolio_mc.sort_values('Sharpe', ascending=False)
    
    mc_final.to_pickle('mc.pickle')

In [5]:
# function to read pickle from mc and display results in chart
def view_mc():
    mc_final = pd.read_pickle('mc.pickle')
    mc_portfolio_vol = mc_final['Volatility'].tolist()
    mc_portfolio_returns = mc_final['Return'].tolist()
    mc_sharpe_ratios = mc_final['Sharpe'].tolist()
    
    fig = plt.figure(figsize=(12,6))
    plt.scatter(mc_portfolio_vol, mc_portfolio_returns, c=mc_sharpe_ratios)
    plt.colorbar(label='Sharpe Ratio')
    plt.xlabel('Volatility')
    plt.ylabel('Returns')
    plt.title('MC Simulation Results')
    plt.show()

In [6]:
# function to read pickle and return top 10
def view_tbl():
    mc_final = pd.read_pickle('mc.pickle')
    # df = mc_final.head(10)
    return mc_final.head(10)    

### Tkinter Display Section

In [1]:
import tkinter as tk
from tkinter import ttk
from pandastable import Table, TableModel

In [3]:
root = tk.Tk()
root.title('Monte Carlo Simulation')

# window size
window_width = 600
window_height = 500

# get overall screen size and center window initially
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()
center_x = int(screen_width/2 - window_width/2)
center_y = int(screen_height/2 - window_height/2)

# window size and initial position
root.geometry(f'{window_width}x{window_height}+{center_x}+{center_y}')

# tk.Label(root, text='Classic Label').pack()
label = ttk.Label(root, 
                  text='Pick a minimum of 2 and a maximum of 10 securities \n sequentially down each entry box',
                 font=('Helvetica',12))
label.pack(ipadx=10, ipady=10)

# put tickers into a list to pass to functions
def get_input():
    sec_list = []
    val0=security0.get("1.0","end-1c")
    val1=security1.get("1.0","end-1c")
    val2=security2.get("1.0","end-1c")
    val3=security3.get("1.0","end-1c")
    val4=security4.get("1.0","end-1c")
    val5=security5.get("1.0","end-1c")
    val6=security6.get("1.0","end-1c")
    val7=security7.get("1.0","end-1c")
    val8=security8.get("1.0","end-1c")
    val9=security9.get("1.0","end-1c")
    sec_list.append(val0)
    sec_list.append(val1)
    sec_list.append(val2)
    sec_list.append(val3)
    sec_list.append(val4)
    sec_list.append(val5)
    sec_list.append(val6)
    sec_list.append(val7)
    sec_list.append(val8)
    sec_list.append(val9)
    return sec_list

# chart of stocks' performance
def perf_chrt():
    sec_list = get_input()
    # print(sec_list)
    
    if sec_list[0] == "":
        err_label = ttk.Label(root, text="Error, box 1 must contain a ticker", font=('Helvetica',10))
    else:
        # run mc sim passing sec_list values
        chart_perf(sec_list)
    
# run monte carlo simulation for tickers    
def mc_sim():
    sec_list = get_input()
    # print(sec_list)
    
    if (sec_list[0] == "") or (sec_list[1] == ""):
        err_label = ttk.Label(root, text="Error, you must pick at least 2 tickers", font=('Helvetica',10))
    else:
        # run mc sim passing sec_list values
        mc_hammer(sec_list)
        # print(sec_list)
        
# view monte carlo sim results chart
def mc_view():
    view_mc()
    
# view top 10 weights in window    
def tbl_view():
        frame = tk.Toplevel(root)
        table = Table(
            frame, dataframe=view_tbl(), showtoolbar=True, showstatusbar=True)
        table.show()

# export top 10 to excel        
def xls_export():
    mc_final = pd.read_pickle('mc.pickle')
    mc_final.to_excel('Top10Weights.xlsx')
    
    
#Creating text box widgets and labels
security0=tk.Text(root, height=1, width=7)
security0.place(x=90, y=80)
sec0=tk.Label(root,text='Security1: ')
sec0.place(x=10, y=80)

security1=tk.Text(root, height=1, width=7)
security1.place(x=90, y=100)
sec1=tk.Label(root,text='Security2: ')
sec1.place(x=10, y=100)

security2=tk.Text(root, height=1, width=7)
security2.place(x=90, y=120)
sec2=tk.Label(root,text='Security3: ')
sec2.place(x=10, y=120)

security3=tk.Text(root, height=1, width=7)
security3.place(x=90, y=140)
sec3=tk.Label(root,text='Security4: ')
sec3.place(x=10, y=140)

security4=tk.Text(root, height=1, width=7)
security4.place(x=90, y=160)
sec4=tk.Label(root,text='Security5: ')
sec4.place(x=10, y=160)

security5=tk.Text(root, height=1, width=7)
security5.place(x=90, y=180)
sec5=tk.Label(root,text='Security6: ')
sec5.place(x=10, y=180)

security6=tk.Text(root, height=1, width=7)
security6.place(x=90, y=200)
sec6=tk.Label(root,text='Security7: ')
sec6.place(x=10, y=200)

security7=tk.Text(root, height=1, width=7)
security7.place(x=90, y=220)
sec7=tk.Label(root,text='Security8: ')
sec7.place(x=10, y=220)

security8=tk.Text(root, height=1, width=7)
security8.place(x=90, y=240)
sec8=tk.Label(root,text='Security9: ')
sec8.place(x=10, y=240)

security9=tk.Text(root, height=1, width=7)
security9.place(x=90, y=260)
sec9=tk.Label(root,text='Security10:')
sec9.place(x=10, y=260)

# Button to view stocks' performance
btn_perf = tk.Button(root, height=3, width=18, text="Chart of Performance", command=lambda: perf_chrt())
btn_perf.place(x=200, y=160)

# Button to run Monte Carlo Simulation
btn_run = tk.Button(root, height=3, width=18, text="Run Simulation", command=lambda: mc_sim())
btn_run.place(x=400, y=160)

# Button to display Monte Carlo sim results
btn_view = tk.Button(root, height=1, width=18, text="View MC Results", command=lambda: mc_view())
btn_view.place(x=400, y=220)

# Button to view top 10 weights in popup window
btn_table = tk.Button(root, height=1, width=18, text="View Top 10 Weights", command=lambda: tbl_view())
btn_table.place(x=400, y=245)

# Button to export top 10 weights to Excel
btn_excel = tk.Button(root, height=1, width=18, text="Excel Export Top 10", command=lambda: xls_export())
btn_excel.place(x=400, y=270)

# launch window
root.mainloop()

In [16]:
class McSim:
    def __init__(self, master):
        self.master = master
        master.title("Monte Carlo Simulation")
        self.button1 = tk.Button(
            master, text="Show Weights", command=self.display_df_in_new_window
        )
        self.button1.place(x=60, y=100, height=44, width=127)

    def get_data(self):
        mc_final = pd.read_pickle('mc.pickle')
        df = mc_final.head(10)
        return df

    def display_df_in_new_window(self):
        frame = tk.Toplevel(self.master)
        self.table = Table(
            frame, dataframe=self.get_data(), showtoolbar=True, showstatusbar=True
        )
        self.table.show()

root = tk.Tk()
my_gui = McSim(root)
root.geometry('600x450')
root.mainloop()